# FastRNN and FastGRNN in Tensorflow

This is a simple notebook that illustrates the usage of Tensorflow implementation of FastRNN and FastGRNN. We are using the USPS dataset. Please refer to `fetch_usps.py` and run it for downloading and cleaning up the dataset.

In [ ]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

import helpermethods
import tensorflow as tf
import numpy as np
import sys
sys.path.insert(0, '../../')

#Provide the GPU number to be used
os.environ['CUDA_VISIBLE_DEVICES'] =''

#FastRNN and FastGRNN imports
from edgeml.trainer.fastTrainer import FastTrainer
from edgeml.graph.rnn import FastGRNNCell
from edgeml.graph.rnn import FastRNNCell

# Fixing seeds for reproducibility
tf.set_random_seed(42)
np.random.seed(42)

# USPS Data

It is assumed that the USPS data has already been downloaded and set up with the help of [fetch_usps.py](fetch_usps.py) and is present in the `./usps10` subdirectory.

Note: Even though usps10 is not a time-series dataset, it can be assumed as, a time-series where each row is coming in at one single time.
So the number of timesteps = 16 and inputDims = 16

In [ ]:
#Loading and Pre-processing dataset for FastCells
dataDir = "usps10/"
(dataDimension, numClasses, Xtrain, Ytrain, Xtest, Ytest) = helpermethods.preProcessData(dataDir)
print("Feature Dimension: ", dataDimension)
print("Num classes: ", numClasses)

# Model Parameters

FastRNN and FastGRNN work for most of the hyper-parameters with which you could acheive decent accuracies on LSTM/GRU. Over and above that, you can use low-rank, sparsity and quatization to reduce model size upto 45x when compared to LSTM/GRU.

In [ ]:
cell = "FastGRNN" # Choose between FastGRNn & FastRNN

inputDims = 16 #features taken in by RNN in one timestep
hiddenDims = 32 #hidden state of RNN

totalEpochs = 300
batchSize = 100

learningRate = 0.01
decayStep = 200
decayRate = 0.1

outFile = None #provide your file, if you need all the logging info in a file

#low-rank parameterisation for weight matrices. None => Full Rank
wRank = None 
uRank = None 

#Sparsity of the weight matrices. x => 100*x % are non-zeros
sW = 1.0 
sU = 1.0

#Non-linearities for the RNN architecture. Can choose from "tanh, sigmoid, relu, quantTanh, quantSigm"
update_non_linearity = "tanh"
gate_non_linearity = "sigmoid"

assert dataDimension % inputDims == 0, "Infeasible per step input, Timesteps have to be integer"

Placeholders for Data feeding during training and infernece

In [ ]:
X = tf.placeholder("float", [None, int(dataDimension / inputDims), inputDims])
Y = tf.placeholder("float", [None, numClasses])

Creating a directory for current model in the datadirectory using timestamp

In [ ]:
currDir = helpermethods.createTimeStampDir(dataDir, cell)
helpermethods.dumpCommand(sys.argv, currDir)

# FastCell Graph Object

Instantiating the FastCell Graph using modular RNN Cells which will be used for training and inference.

Note: RNN cells in edgeml.rnn can be used anywhere in place of LSTM/GRU in a plug & play fashion.

In [ ]:
#Create appropriate RNN cell object based on choice
if cell == "FastGRNN":
    FastCell = FastGRNNCell(hiddenDims, gate_non_linearity=gate_non_linearity,
                            update_non_linearity=update_non_linearity,
                            wRank=wRank, uRank=uRank)
elif cell == "FastRNN":
    FastCell = FastRNNCell(hiddenDims, update_non_linearity=update_non_linearity,
                           wRank=wRank, uRank=uRank)
else:
    sys.exit('Exiting: No Such Cell as ' + cell)

# FastCell Trainer Object

Instantiating the FastCell Trainer which will be used for 3 phase training

In [ ]:
FastCellTrainer = FastTrainer(FastCell, X, Y, sW=sW, sU=sU, learningRate=learningRate, outFile=outFile)

Session declaration and variable initialization. Interactive Session doesn't clog the entire GPU.

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# FastCell Training Routine

The method to to run the 3 phase training, followed by giving out the best early stopping model, accuracy along with saving of the parameters.

In [ ]:
FastCellTrainer.train(batchSize, totalEpochs, sess, Xtrain, Xtest,
                      Ytrain, Ytest, decayStep, decayRate, dataDir, currDir)

If one uses piece-wise linear approximations for non-linearities liek quant Tanh for tanh and quantSigm for Sigmoid, they can benefit greatly from pure integer arithmetic after model quantization during prediction

In [1]:
#Model quantization
model_dir = "" #you wil lsee model dir printed at the end of trianing

import quantizeFastModels
quantizeFastModels.quantizeFastModels(model_dir)